# Sudoku-Extreme Training with Alternative Depth Controllers

This notebook trains the HybridPoHHRMSolver on Sudoku-Extreme with different depth controllers:
- **LSTM Controller** - Stronger gating than GRU
- **Transformer Controller** - Causal attention over depth history

## Features
- On-the-fly Sudoku augmentation (digit permutation, transpose, row/col shuffling)
- W&B logging (optional)
- Checkpoint saving
- Cosine LR schedule

## 1. Setup & Imports

In [ ]:
# Install dependencies if needed (uncomment if running on Colab)
# !pip install torch wandb tqdm

In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

# Add project root to path
PROJECT_ROOT = Path(".").resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

In [ ]:
from src.pot.models.sudoku_solver import HybridPoHHRMSolver

## 2. Configuration

In [ ]:
# =============================================================================
# CONFIGURATION - Modify these settings as needed
# =============================================================================

config = {
    # Controller type: "lstm", "transformer", "gru", "xlstm", "mingru"
    "controller_type": "lstm",  # <-- CHANGE THIS TO SWITCH CONTROLLERS
    
    # Data
    "data_dir": "../data/sudoku-extreme",
    
    # Model architecture
    "d_model": 512,
    "n_heads": 8,
    "h_layers": 2,
    "l_layers": 2,
    "h_cycles": 2,
    "l_cycles": 8,
    "halt_max_steps": 4,
    
    # Transformer controller specific (only used if controller_type="transformer")
    "d_ctrl": 256,
    "n_ctrl_layers": 2,
    "n_ctrl_heads": 4,
    "max_depth": 32,
    
    # Training
    "epochs": 1000,
    "batch_size": 768,
    "lr": 1e-4,
    "weight_decay": 0.1,
    "warmup_steps": 1000,
    "augment": True,  # On-the-fly Sudoku augmentation
    
    # Device
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    
    # Logging
    "use_wandb": False,  # Set to True to enable W&B logging
    "wandb_project": "sudoku-controllers",
    
    # Checkpoints
    "save_dir": "../checkpoints",
    "save_every": 50,
}

print("Configuration:")
for k, v in config.items():
    print(f"  {k}: {v}")

## 3. Sudoku Augmentation

In [ ]:
def shuffle_sudoku(board: np.ndarray, solution: np.ndarray):
    """
    Apply validity-preserving augmentation to a Sudoku puzzle.
    
    Transforms include:
    - Digit permutation (1-9 -> random permutation)
    - Transpose (50% chance)
    - Row band shuffling (shuffle the 3 bands of 3 rows each)
    - Row shuffling within bands
    - Column stack shuffling (shuffle the 3 stacks of 3 columns each)  
    - Column shuffling within stacks
    
    Args:
        board: Input puzzle [81] with 0=blank, 1-9=digits
        solution: Solution [81] with 1-9
        
    Returns:
        Augmented (board, solution) tuple
    """
    # Create a random digit mapping: a permutation of 1..9, with zero (blank) unchanged
    digit_map = np.pad(np.random.permutation(np.arange(1, 10)), (1, 0))
    
    # Randomly decide whether to transpose
    transpose_flag = np.random.rand() < 0.5

    # Generate a valid row permutation:
    # - Shuffle the 3 bands (each band = 3 rows) and for each band, shuffle its 3 rows.
    bands = np.random.permutation(3)
    row_perm = np.concatenate([b * 3 + np.random.permutation(3) for b in bands])

    # Similarly for columns (stacks).
    stacks = np.random.permutation(3)
    col_perm = np.concatenate([s * 3 + np.random.permutation(3) for s in stacks])

    # Build an 81->81 mapping
    mapping = np.array([row_perm[i // 9] * 9 + col_perm[i % 9] for i in range(81)])

    def apply_transformation(x: np.ndarray) -> np.ndarray:
        # Reshape to 9x9 for transpose
        x_2d = x.reshape(9, 9)
        if transpose_flag:
            x_2d = x_2d.T
        x_flat = x_2d.flatten()
        # Apply row/col permutation
        new_board = x_flat[mapping]
        # Apply digit mapping
        return digit_map[new_board]

    return apply_transformation(board), apply_transformation(solution)


class SudokuDataset(Dataset):
    """Sudoku dataset with on-the-fly augmentation."""
    
    def __init__(self, inputs, solutions, puzzle_ids=None, augment=True):
        self.inputs = inputs
        self.solutions = solutions
        self.puzzle_ids = puzzle_ids if puzzle_ids is not None else torch.zeros(len(inputs), dtype=torch.long)
        self.augment = augment
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        inp = self.inputs[idx].numpy() if isinstance(self.inputs[idx], torch.Tensor) else self.inputs[idx]
        sol = self.solutions[idx].numpy() if isinstance(self.solutions[idx], torch.Tensor) else self.solutions[idx]
        pid = self.puzzle_ids[idx]
        
        if self.augment:
            inp, sol = shuffle_sudoku(inp, sol)
        
        return torch.tensor(inp, dtype=torch.long), torch.tensor(sol, dtype=torch.long), pid

## 4. Data Loading

In [ ]:
def load_sudoku_data(data_dir: str):
    """
    Load Sudoku-Extreme dataset.
    
    Expected format: .pt files with 'inputs' and 'solutions' tensors.
    """
    data_path = Path(data_dir)
    
    train_data = {}
    val_data = {}
    
    # Try to load preprocessed .pt files
    train_pt = data_path / "train.pt"
    val_pt = data_path / "val.pt"
    
    if train_pt.exists() and val_pt.exists():
        print(f"Loading preprocessed data from {data_dir}...")
        train_data = torch.load(train_pt, map_location="cpu")
        val_data = torch.load(val_pt, map_location="cpu")
        print(f"  Train samples: {len(train_data['inputs'])}")
        print(f"  Val samples: {len(val_data['inputs'])}")
    else:
        # Create synthetic data for testing
        print("⚠️ No preprocessed data found. Creating synthetic data for testing...")
        print("   Place train.pt and val.pt in the data directory for real training.")
        n_train, n_val = 1000, 200
        train_data = {
            'inputs': torch.randint(0, 10, (n_train, 81)),
            'solutions': torch.randint(1, 10, (n_train, 81)),
            'puzzle_ids': torch.zeros(n_train, dtype=torch.long),
        }
        val_data = {
            'inputs': torch.randint(0, 10, (n_val, 81)),
            'solutions': torch.randint(1, 10, (n_val, 81)),
            'puzzle_ids': torch.zeros(n_val, dtype=torch.long),
        }
    
    return train_data, val_data


def create_dataloaders(train_data, val_data, batch_size: int, augment: bool = True):
    """Create DataLoaders from data dicts with on-the-fly augmentation."""
    
    # Get or create puzzle IDs
    train_ids = train_data.get('puzzle_ids', torch.zeros(len(train_data['inputs']), dtype=torch.long))
    val_ids = val_data.get('puzzle_ids', torch.zeros(len(val_data['inputs']), dtype=torch.long))
    
    # Use SudokuDataset with augmentation for training
    train_dataset = SudokuDataset(
        train_data['inputs'],
        train_data['solutions'],
        train_ids,
        augment=augment,
    )
    
    # No augmentation for validation
    val_dataset = SudokuDataset(
        val_data['inputs'],
        val_data['solutions'],
        val_ids,
        augment=False,
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    return train_loader, val_loader

In [ ]:
# Load data
train_data, val_data = load_sudoku_data(config["data_dir"])
train_loader, val_loader = create_dataloaders(
    train_data, val_data, 
    batch_size=config["batch_size"],
    augment=config["augment"]
)

print(f"\nDataLoaders created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")

## 5. Model Creation

In [ ]:
def create_model(config):
    """Create HybridPoHHRMSolver with specified controller type."""
    
    controller_type = config["controller_type"]
    
    # Build controller kwargs based on type
    if controller_type == "transformer":
        controller_kwargs = {
            "d_ctrl": config["d_ctrl"],
            "n_ctrl_layers": config["n_ctrl_layers"],
            "n_ctrl_heads": config["n_ctrl_heads"],
            "max_depth": config["max_depth"],
            "token_conditioned": True,
        }
    else:
        # LSTM, GRU, xLSTM, minGRU
        controller_kwargs = {
            "d_ctrl": config["d_model"],
            "token_conditioned": True,
        }
    
    model = HybridPoHHRMSolver(
        d_model=config["d_model"],
        n_heads=config["n_heads"],
        H_layers=config["h_layers"],
        L_layers=config["l_layers"],
        H_cycles=config["h_cycles"],
        L_cycles=config["l_cycles"],
        halt_max_steps=config["halt_max_steps"],
        controller_type=controller_type,
        controller_kwargs=controller_kwargs,
    )
    
    return model

In [ ]:
# Create model
device = config["device"]
model = create_model(config).to(device)

param_count = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model created on {device}")
print(f"  Controller: {config['controller_type'].upper()}")
print(f"  Total parameters: {param_count:,}")
print(f"  Trainable parameters: {trainable_params:,}")

## 6. Training Functions

In [ ]:
def train_epoch(model, dataloader, optimizer, device, epoch):
    """Train for one epoch."""
    model.train()
    total_loss = 0
    total_correct = 0
    total_cells = 0
    total_grids_correct = 0
    total_grids = 0
    
    pbar = tqdm(dataloader, desc=f"Epoch {epoch}", leave=False)
    for inputs, targets, puzzle_ids in pbar:
        inputs = inputs.to(device)
        targets = targets.to(device)
        puzzle_ids = puzzle_ids.to(device)
        
        optimizer.zero_grad()
        
        # Forward
        outputs = model(inputs, puzzle_ids)
        logits = outputs[0]  # [B, 81, 10]
        
        # Loss
        loss = nn.functional.cross_entropy(
            logits.view(-1, 10),
            targets.view(-1),
        )
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        # Metrics
        preds = logits.argmax(dim=-1)
        correct = (preds == targets).sum().item()
        total_correct += correct
        total_cells += targets.numel()
        
        # Grid accuracy
        grid_correct = (preds == targets).all(dim=1).sum().item()
        total_grids_correct += grid_correct
        total_grids += targets.size(0)
        
        total_loss += loss.item()
        
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'cell': f'{100*correct/targets.numel():.1f}%',
            'grid': f'{100*grid_correct/targets.size(0):.1f}%',
        })
    
    return {
        'loss': total_loss / len(dataloader),
        'cell_acc': total_correct / total_cells,
        'grid_acc': total_grids_correct / total_grids,
    }


@torch.no_grad()
def evaluate(model, dataloader, device):
    """Evaluate model."""
    model.eval()
    total_loss = 0
    total_correct = 0
    total_cells = 0
    total_grids_correct = 0
    total_grids = 0
    
    for inputs, targets, puzzle_ids in tqdm(dataloader, desc="Evaluating", leave=False):
        inputs = inputs.to(device)
        targets = targets.to(device)
        puzzle_ids = puzzle_ids.to(device)
        
        outputs = model(inputs, puzzle_ids)
        logits = outputs[0]
        
        loss = nn.functional.cross_entropy(
            logits.view(-1, 10),
            targets.view(-1),
        )
        
        preds = logits.argmax(dim=-1)
        total_correct += (preds == targets).sum().item()
        total_cells += targets.numel()
        total_grids_correct += (preds == targets).all(dim=1).sum().item()
        total_grids += targets.size(0)
        total_loss += loss.item()
    
    return {
        'loss': total_loss / len(dataloader),
        'cell_acc': total_correct / total_cells,
        'grid_acc': total_grids_correct / total_grids,
    }

## 7. Setup Optimizer & Scheduler

In [ ]:
# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=config["lr"],
    weight_decay=config["weight_decay"],
    betas=(0.9, 0.95),
)

# Scheduler (cosine annealing)
total_steps = config["epochs"] * len(train_loader)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)

print(f"Optimizer: AdamW (lr={config['lr']}, weight_decay={config['weight_decay']})")
print(f"Scheduler: CosineAnnealingLR (T_max={total_steps})")

## 8. W&B Setup (Optional)

In [ ]:
# Initialize W&B if enabled
if config["use_wandb"]:
    import wandb
    
    run_name = f"{config['controller_type']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    wandb.init(
        project=config["wandb_project"],
        name=run_name,
        config=config,
    )
    wandb.watch(model, log="gradients", log_freq=100)
    print(f"W&B initialized: {run_name}")
else:
    print("W&B logging disabled. Set config['use_wandb'] = True to enable.")

## 9. Training Loop

In [ ]:
# Create checkpoint directory
save_dir = Path(config["save_dir"]) / config["controller_type"]
save_dir.mkdir(parents=True, exist_ok=True)
print(f"Checkpoints will be saved to: {save_dir}")

In [ ]:
# Training history
history = {
    'train_loss': [],
    'train_cell_acc': [],
    'train_grid_acc': [],
    'val_loss': [],
    'val_cell_acc': [],
    'val_grid_acc': [],
    'lr': [],
}

best_grid_acc = 0

print(f"\n{'='*60}")
print(f"Starting {config['controller_type'].upper()} Controller Training")
print(f"{'='*60}\n")

In [ ]:
# Main training loop
for epoch in range(1, config["epochs"] + 1):
    # Train
    train_metrics = train_epoch(model, train_loader, optimizer, device, epoch)
    
    # Evaluate
    val_metrics = evaluate(model, val_loader, device)
    
    # Update scheduler
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    
    # Store history
    history['train_loss'].append(train_metrics['loss'])
    history['train_cell_acc'].append(train_metrics['cell_acc'])
    history['train_grid_acc'].append(train_metrics['grid_acc'])
    history['val_loss'].append(val_metrics['loss'])
    history['val_cell_acc'].append(val_metrics['cell_acc'])
    history['val_grid_acc'].append(val_metrics['grid_acc'])
    history['lr'].append(current_lr)
    
    # Print progress
    print(f"Epoch {epoch}/{config['epochs']} | "
          f"Train: loss={train_metrics['loss']:.4f}, cell={100*train_metrics['cell_acc']:.1f}%, grid={100*train_metrics['grid_acc']:.1f}% | "
          f"Val: loss={val_metrics['loss']:.4f}, cell={100*val_metrics['cell_acc']:.1f}%, grid={100*val_metrics['grid_acc']:.1f}%")
    
    # W&B logging
    if config["use_wandb"]:
        wandb.log({
            "epoch": epoch,
            "train/loss": train_metrics['loss'],
            "train/cell_acc": train_metrics['cell_acc'],
            "train/grid_acc": train_metrics['grid_acc'],
            "val/loss": val_metrics['loss'],
            "val/cell_acc": val_metrics['cell_acc'],
            "val/grid_acc": val_metrics['grid_acc'],
            "lr": current_lr,
        })
    
    # Save best model
    if val_metrics['grid_acc'] > best_grid_acc:
        best_grid_acc = val_metrics['grid_acc']
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_grid_acc': best_grid_acc,
            'config': config,
        }, save_dir / "best_model.pt")
        print(f"  ✓ New best! Grid acc: {100*best_grid_acc:.2f}%")
    
    # Periodic checkpoint
    if epoch % config["save_every"] == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'grid_acc': val_metrics['grid_acc'],
            'config': config,
        }, save_dir / f"checkpoint_epoch{epoch}.pt")
        print(f"  💾 Checkpoint saved: epoch {epoch}")

print(f"\n{'='*60}")
print(f"Training complete! Best grid accuracy: {100*best_grid_acc:.2f}%")
print(f"{'='*60}")

if config["use_wandb"]:
    wandb.finish()

## 10. Training Visualization

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Cell Accuracy
axes[1].plot([100*x for x in history['train_cell_acc']], label='Train')
axes[1].plot([100*x for x in history['val_cell_acc']], label='Val')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Cell Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Grid Accuracy
axes[2].plot([100*x for x in history['train_grid_acc']], label='Train')
axes[2].plot([100*x for x in history['val_grid_acc']], label='Val')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Accuracy (%)')
axes[2].set_title('Grid Accuracy')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.suptitle(f"{config['controller_type'].upper()} Controller Training", fontsize=14)
plt.tight_layout()
plt.savefig(save_dir / "training_curves.png", dpi=150)
plt.show()

print(f"\nPlot saved to: {save_dir / 'training_curves.png'}")

## 11. Load Best Model & Final Evaluation

In [ ]:
# Load best model
checkpoint = torch.load(save_dir / "best_model.pt", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

print(f"Loaded best model from epoch {checkpoint['epoch']}")
print(f"Best grid accuracy: {100*checkpoint['best_grid_acc']:.2f}%")

In [ ]:
# Final evaluation
final_metrics = evaluate(model, val_loader, device)

print(f"\n{'='*40}")
print(f"Final Evaluation Results")
print(f"{'='*40}")
print(f"Controller: {config['controller_type'].upper()}")
print(f"Val Loss: {final_metrics['loss']:.4f}")
print(f"Cell Accuracy: {100*final_metrics['cell_acc']:.2f}%")
print(f"Grid Accuracy: {100*final_metrics['grid_acc']:.2f}%")
print(f"{'='*40}")